In [1]:
#Gesture Volume Control

In [2]:
#importing libraries
import cv2
import time
import math
import numpy as np
import mediapipe as mp
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [3]:
#required variables
previous_time=0
current_time=0

index_x=0 
index_y=0

thumb_x=0
thumb_y=0

vol=0
abs_vol=0

lengths=[]

In [4]:
#audio library initialization
devices=AudioUtilities.GetSpeakers()
interface=devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume=cast(interface, POINTER(IAudioEndpointVolume))
range=volume.GetVolumeRange()

minVol=range[0]
maxVol=range[1]

#capturing the video
cap=cv2.VideoCapture(1)

#this will help us detect the hand and draw the reference points
mphand=mp.solutions.hands
hands=mphand.Hands(max_num_hands=1, min_detection_confidence=0.9)
mpDraw=mp.solutions.drawing_utils    #this statement will help us plot the 21 reference points and their connections

while True:
    success, img=cap.read()
    
    #converting image to rgb
    img_rgb=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    #detecting hand/s using 21 points of reference, these reference points will be plotted on the hand
    result=hands.process(img_rgb)
    if result.multi_hand_landmarks:
        for handlms in result.multi_hand_landmarks:    #loop: handlms gives us the number of hands i.e. 1 hand, 2hands...
            for id, lm in enumerate(handlms.landmark):    #loop: gives us individual reference point on each hand
                h, w, c=img.shape                         #gives us the dimensions of the frame
                cx, cy=int(lm.x*w), int(lm.y*h)           #give us the coordinates for the reference points
                if id==8:
                    cv2.circle(img, (cx, cy), 20, (255, 0, 255), cv2.FILLED)
                    index_x, index_y=cx, cy
                elif id==4:
                    cv2.circle(img, (cx, cy), 20, (255, 0, 255), cv2.FILLED)
                    thumb_x, thumb_y=cx, cy
                cv2.line(img, (index_x, index_y), (thumb_x, thumb_y), (255, 0, 255), 3)
                length=math.dist((index_x, index_y), (thumb_x, thumb_y))
                lengths.append(length)
                vol=np.interp(length, [50, 300], [minVol, maxVol])
                abs_vol=np.interp(vol, [minVol, maxVol], [0, 100])
                volume.SetMasterVolumeLevel(vol, None)
            mpDraw.draw_landmarks(img, handlms, mphand.HAND_CONNECTIONS)
            #the above statement will actually plot the 21 reference points and their connections
    
    #calculating the frame rate
    current_time=time.time()
    frame_rate=1/(current_time-previous_time)
    previous_time=current_time
    
    img=cv2.flip(img,1)
    #cv2.putText(img, str(int(frame_rate)), (20, 40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
    cv2.putText(img, str(int(abs_vol)), (20, 40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break